In [81]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
import math
from PIL import Image
%matplotlib inline

In [82]:
def camera_calibation():
    images = glob.glob('./camera_cal/calibration*.jpg')

    object_points = []
    image_points = []

    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    for fname in images:
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        if ret == True:
            image_points.append(corners)
            object_points.append(objp)

            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

    img = cv2.imread('./camera_cal/calibration3.jpg')
    img_size = (img.shape[1], img.shape[0])

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img_size, None, None)
    return (mtx, dist)

global calibration
calibration = camera_calibation()

In [157]:
class Line():
    def __init__(self):
        self.recent_amount = 7
        self.weights = [.2,.2,.2,.1,.1,.1,.1]
        self.detected = False
        self.recent_xfitted = []
        self.bestx = None
        self.recent_fits = []
        self.best_fit = None
        self.current_fit = [np.array([False])]
        self.radius_of_curvature = None
        self.line_base_pos = None
        self.allx = None
        self.ally = None

    def preliminary_update(self, line_fit, fitx):
        if line_fit is not None:
            
            if self.should_update(fitx):
                self.detected = True
                self.current_fit = line_fit
                self.allx = fitx
                if self.best_fit is None:
                    self.update()
            else:
                self.detected = False
    
    def should_update(self, new_fitx):
        should_update = True
        
        if  self.bestx is not None:
            
            if new_fitx[719] > self.bestx[719] + 50 or new_fitx[719] < self.bestx[719] - 50:
                should_update = False

        return should_update
    
    def get_radius(self, line):
        ploty = np.linspace(0, 719, num=720)
        y_eval = np.max(ploty)
        curverad = ((1 + (2*line[0]*y_eval + line[1])**2)**1.5) / np.absolute(2*line[0])
        ym_per_pix = 3/170
        xm_per_pix = 3.7/210
        fit_cr = np.polyfit(ploty*ym_per_pix, line*xm_per_pix, 2)
        curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
        return curverad
    
    def update(self):
        self.detected = True
        self.recent_fits.append(self.current_fit)
        self.recent_fits = self.recent_fits[-self.recent_amount:]
        self.best_fit = np.average(self.recent_fits, axis=0, weights=self.weights[:len(self.recent_fits)]).astype(int)
        self.recent_xfitted.append(self.allx)
        self.recent_xfitted = self.recent_xfitted[-self.recent_amount:]
        self.bestx = np.average(self.recent_xfitted, axis=0, weights=self.weights[:len(self.recent_xfitted)]).astype(int)

In [180]:
def undistort(img, mtx, dist):
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    return undistorted_img

def get_warp_matrix(img):
    img_size = (int(img.shape[1]/2), img.shape[0])
    bot_width = 1.3
    mid_width = .30
    height_pct = .665
    bottom_trim = .945
    src = np.float32([
            [img.shape[1]*(.5-mid_width/2), img.shape[0]*height_pct],
            [img.shape[1]*(.5+mid_width/2), img.shape[0]*height_pct], 
            [img.shape[1]*(.5+bot_width/2), img.shape[0]*bottom_trim], 
            [img.shape[1]*(.5-bot_width/2), img.shape[0]*bottom_trim]
    ])
    offset = img_size[0]*.15
    dst = np.float32([[offset, 0], [img_size[0] - offset, 0], [img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src,dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return M, M_inv

def threshold_image(img):
    kernel = np.ones((14,14),np.uint8)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS) 
    
    hls_l = hls[:,:,1]
    th_hls_l = cv2.morphologyEx(hls_l, cv2.MORPH_TOPHAT, kernel)
    hls_l_binary = np.zeros_like(th_hls_l)
    hls_l_binary[(th_hls_l > 20) & (th_hls_l <= 255)] = 1

    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB) 
    lab_l = lab[:,:,0]
    th_lab_l = cv2.morphologyEx(lab_l, cv2.MORPH_TOPHAT, kernel)
    lab_l_binary = np.zeros_like(th_lab_l)
    lab_l_binary[(th_lab_l > 20) & (th_lab_l <= 255)] = 1

    lab_b = lab[:,:,2]
    th_lab_b = cv2.morphologyEx(lab_b, cv2.MORPH_TOPHAT, kernel)
    lab_b_binary = np.zeros_like(th_lab_b)
    lab_b_binary[(th_lab_b > 5) & (th_lab_b <= 255)] = 1

    full_mask = np.zeros_like(th_hls_l)
    full_mask[(hls_l_binary == 1) | (lab_l_binary == 1) | (lab_b_binary == 1)] = 1

    kernel = np.ones((6,3),np.uint8)
    erosion = cv2.erode(full_mask,kernel,iterations = 1)

    return erosion

def locate_line(thresholded, lines, lane_side):
    histogram = np.sum(thresholded[thresholded.shape[0]/2:,:], axis=0)
    out_img = np.dstack((thresholded, thresholded, thresholded))*255
    midpoint = np.int(histogram.shape[0]/2)
   
    if lane_side == 'left':
        x_base = np.argmax(histogram[:midpoint])
    else:
        x_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 9
    window_height = np.int(thresholded.shape[0]/nwindows)
    nonzero = thresholded.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    x_current = x_base
    margin = 30
    minpix = 50
    lane_inds = []

    for window in range(nwindows):
        win_y_low = thresholded.shape[0] - (window+1)*window_height
        win_y_high = thresholded.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),(0,255,0), 2)
        good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_x_low) & (nonzerox < win_x_high)).nonzero()[0]
        lane_inds.append(good_inds)
        if len(good_inds) > minpix:
            x_current = np.int(np.mean(nonzerox[good_inds]))
            
    lane_inds = np.concatenate(lane_inds)

    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds]
    
    if x.shape[0] != 0:
        line_fit = np.polyfit(y, x, 2)
        ploty = np.linspace(0, thresholded.shape[0]-1, thresholded.shape[0] )
        fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]
        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]
        radius = get_radius(fitx, ploty)
        lines[lane_side].preliminary_update(line_fit, fitx)
    
    return lines

def line_in_windows(thresholded, line):
    line_fit = fitx = radius = None
    fit = line.current_fit
    out_img = np.dstack((thresholded, thresholded, thresholded))*255
    nonzero = thresholded.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 30
    lane_inds = ((nonzerox > (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] - margin)) & (nonzerox < (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] + margin))) 
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    if x.shape[0] != 0:
        line_fit = np.polyfit(y, x, 2)
        ploty = np.linspace(0, thresholded.shape[0]-1, thresholded.shape[0] )
        fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]
        radius = get_radius(fitx, ploty)
        line.preliminary_update(line_fit, fitx)
    
    return line

def get_radius(line, ploty):
    y_eval = np.max(ploty)
    curverad = ((1 + (2*line[0]*y_eval + line[1])**2)**1.5) / np.absolute(2*line[0])
    ym_per_pix = 3/170
    xm_per_pix = 3.7/210
    fit_cr = np.polyfit(ploty*ym_per_pix, line*xm_per_pix, 2)
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    return curverad
    
def get_overlayer(warped, lines):
    if lines is not None:
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        pts_left = np.array([np.transpose(np.vstack([lines['left'].bestx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([lines['right'].bestx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        overlayer = cv2.fillPoly(warp_zero, np.int_([pts]), (0,255, 0))
    return overlayer

def get_line_fit(thresholded_image, lines, side):
    
    if lines[side].detected == True:
        lines[side] = line_in_windows(thresholded_image, lines[side])
    
    if lines[side].detected == False:
        lines = locate_line(thresholded_image, lines, side)
        
    return lines[side]


def confirm_angle(left, right):
    ret_value = True

    if left.current_fit[1] > right.current_fit[1] + .6:
        ret_value = False
        
    if left.current_fit[1] < right.current_fit[1] -.6:
        ret_value = False
        
    return ret_value

def confirm_spacing(left, right):
    ret_value = True
    new_space = right.allx[719] - left.allx[719]
    average_space = right.bestx[719] - left.bestx[719]

    if new_space > average_space + 100 or new_space < average_space - 100:
        
        ret_value = False
        
    return ret_value

def confirm_curve(left, right):
    ret_value = True

    if left.current_fit[0] > right.current_fit[0] + .0007:#was .0005
        ret_value = False
        
    if left.current_fit[0] < right.current_fit[0] - .0007:# was .0005
        ret_value = False

    return ret_value

def get_left_right_compliance(target, compare):
    spacing_confirmed = confirm_spacing(target, compare)
    angle_confirmed = confirm_angle(target, compare)
    curve_confirmed = confirm_curve(target, compare)
    return spacing_confirmed and angle_confirmed and curve_confirmed

def get_steer_wheel(curve):
    global steering_wheel
    angle = (4 * (math.pi + math.asin(9/(curve*2)))) * 200.40
    return steering_wheel.rotate(angle)

def overlay_image(original, overlayer):
    global steering_wheel
    overlayed = cv2.addWeighted(original, 1, overlayer, 0.3, 0)

    ploty = np.linspace(0, 719, num=720)
    y_eval = np.max(ploty)
    img = np.zeros_like(original)

    ym_per_pix = 3/170
    xm_per_pix = 3.7/210
    leftx = lines['left'].bestx
    rightx = lines['right'].bestx

    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)

    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    road_curve = left_curverad + right_curverad / 2
    road_curve = round(road_curve, 2)
    curve_string = "{:,}".format(road_curve)

    if road_curve > 3000:
        curve_string = 'Straight'
    else:
        curve_string = str(curve_string) + 'm'

    center_of_lines = (rightx[len(rightx)-1] + leftx[len(leftx)-1]) / 2
    distance_from_center = center_of_lines - img.shape[1] / 4
    distance_from_center = distance_from_center * xm_per_pix
    distance_from_center = round(distance_from_center, 2)
    center_offset = "{:,}".format(abs(distance_from_center))

    if distance_from_center > 0:
        center_offset = str(center_offset) + 'm left'
    else:
        center_offset = str(center_offset) + 'm right'

    font = cv2.FONT_HERSHEY_SIMPLEX
    image = cv2.putText(overlayed,'Road Curve: ' + curve_string,(10,50), font, 1.4,(255,255,255),2)
    image = cv2.putText(image, 'Off Center: ' + center_offset,(10,100), font, 1.4,(255,255,255),2)

    if ((left_fit_cr[0] + right_fit_cr[0]) / 2) > 0:
        road_curve = -1 * road_curve
    
    background = Image.fromarray(image).convert("RGBA")
    background.paste(get_steer_wheel(road_curve), (540, 150), get_steer_wheel(road_curve))
    image = np.array(background)
    image = image[:,:,:3]
    
    return image

def pipeline(img):
    global lines, calibration
    
    undistorted_img = undistort(img, calibration[0], calibration[1])
    size_for_warp = (int(undistorted_img.shape[1]/2), undistorted_img.shape[0])
    M, M_inv = get_warp_matrix(undistorted_img)
    warped = cv2.warpPerspective(undistorted_img, M, size_for_warp, flags=cv2.INTER_LINEAR)
    
    thresholded_image = threshold_image(warped)

    left = get_line_fit(thresholded_image, lines, 'left')
    right = get_line_fit(thresholded_image, lines, 'right')
    
    if get_left_right_compliance(left, right) and left.detected:
        left.update()

    if get_left_right_compliance(right, left) and right.detected:  
        right.update()

    overlayer = get_overlayer(warped, lines)
    size_for_unwarp = (int(undistorted_img.shape[1]), undistorted_img.shape[0])
    unwarped = cv2.warpPerspective(overlayer, M_inv, size_for_unwarp, flags=cv2.INTER_LINEAR)
    return overlay_image(undistorted_img, unwarped)


In [181]:
def reset_lines():
    left_line = Line()
    right_line = Line()

    global lines
    lines = {
        'left': left_line, 
        'right': right_line
    }
    
    global steering_wheel
    steering_wheel = Image.open('output_images/steering_wheel.png').convert("RGBA")
    steering_wheel= steering_wheel.resize((200,200),Image.ANTIALIAS)

In [182]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [183]:
reset_lines()
first_output = 'first.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(first_output, audio=False)

updated!
updated!
spacings - True
209.842453522
210
angles - True
0.027416893056
0.0591317692402
curves - True
-1.60282166927e-05
-4.86821098879e-05
updated!
spacings - True
-209.842453522
-210
angles - True
0.0591317692402
0.027416893056
curves - True
-4.86821098879e-05
-1.60282166927e-05
updated!
[MoviePy] >>>> Building video first.mp4
[MoviePy] Writing video first.mp4








  0%|          | 0/1261 [00:00<?, ?it/s]





  0%|          | 1/1261 [00:00<03:03,  6.85it/s]

same line space - True
237.343713993
237
False
False
same line space - True
447.186167516
447
False
False
spacings - True
209.842453522
210
angles - True
0.027416893056
0.0591317692402
curves - True
-1.60282166927e-05
-4.86821098879e-05
updated!
spacings - True
-209.842453522
-210
angles - True
0.0591317692402
0.027416893056
curves - True
-4.86821098879e-05
-1.60282166927e-05
updated!


same line space - True
237.003678379
237
False
False
same line space - True
449.352398737
447
False
False
spacings - True
212.348720358
210
angles - True
0.0334909332346
0.0514310656536
curves - True
-2.42243181494e-05
-3.26713046807e-05
updated!
spacings - True
-212.348720358
-210
angles - True
0.0514310656536
0.0334909332346
curves - True
-3.26713046807e-05
-2.42243181494e-05
updated!








  0%|          | 2/1261 [00:00<03:02,  6.90it/s]

...

same line space - True
243.177380554
245
False
False
same line space - True
457.057816732
458
False
False
spacings - True
213.880436179
213
angles - True
-0.0106770346495
-0.00477048118677
curves - True
9.74726841277e-06
1.84857432915e-06
updated!
spacings - True
-213.880436179
-214
angles - True
-0.00477048118677
-0.0106770346495
curves - True
1.84857432915e-06
9.74726841277e-06
updated!
same line space - True
242.935364071
244
False
False
same line space - True
454.913859916
458
False
False
spacings - True
211.978495845
214
angles - True
-0.00873408618657
0.0134016502812
curves - True
7.12330687265e-06
-1.55929344373e-05
updated!
spacings - True
-211.978495845
-214
angles - True
0.0134016502812
-0.00873408618657
curves - True
-1.55929344373e-05
7.12330687265e-06
updated!








100%|█████████▉| 1258/1261 [03:05<00:00,  6.57it/s]





100%|█████████▉| 1259/1261 [03:05<00:00,  6.72it/s]

same line space - True
242.228436956
244
False
False
same line space - True
456.221018392
458
False
False
spacings - True
213.992581436
214
angles - True
-0.00921776726146
0.00716227766288
curves - True
1.02552563389e-05
-7.7022889813e-06
updated!
spacings - True
-213.992581436
-215
angles - True
0.00716227766288
-0.00921776726146
curves - True
-7.7022889813e-06
1.02552563389e-05
updated!
same line space - True
241.659130375
243
False
False
same line space - True
456.727013888
458
False
False
spacings - True
215.067883513
215
angles - True
-0.00851222045391
0.00583164380285
curves - True
1.0191028831e-05
-2.94084083052e-06
updated!
spacings - True
-215.067883513
-215
angles - True
0.00583164380285
-0.00851222045391
curves - True
-2.94084083052e-06
1.0191028831e-05
updated!








100%|█████████▉| 1260/1261 [03:05<00:00,  6.79it/s]







[MoviePy] Done.
[MoviePy] >>>> Video ready: first.mp4 



In [184]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(first_output))

In [185]:
reset_lines()
challenge_output = 'challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(challenge_output, audio=False)

updated!
updated!
spacings - True
195.59979568
196
angles - True
-0.115102267157
0.002609866655
curves - True
3.42674914263e-05
1.01197740559e-05
updated!
spacings - True
-195.59979568
-196
angles - True
0.002609866655
-0.115102267157
curves - True
1.01197740559e-05
3.42674914263e-05
updated!
[MoviePy] >>>> Building video challenge.mp4
[MoviePy] Writing video challenge.mp4








  0%|          | 0/485 [00:00<?, ?it/s]





  0%|          | 1/485 [00:00<01:06,  7.26it/s]

same line space - True
238.89698505
238
False
False
same line space - True
434.49678073
434
False
False
spacings - True
195.59979568
196
angles - True
-0.115102267157
0.002609866655
curves - True
3.42674914263e-05
1.01197740559e-05
updated!
spacings - True
-195.59979568
-196
angles - True
0.002609866655
-0.115102267157
curves - True
1.01197740559e-05
3.42674914263e-05
updated!


same line space - True
238.538561013
238
False
False
same line space - True
434.68181466
434
False
False
spacings - True
196.143253646
196
angles - True
-0.113597041241
0.00290039168732
curves - True
3.1652527227e-05
1.19885797042e-05
updated!
spacings - True
-196.143253646
-196
angles - True
0.00290039168732
-0.113597041241
curves - True
1.19885797042e-05
3.1652527227e-05
updated!








  0%|          | 2/485 [00:00<01:08,  7.07it/s]

...







100%|█████████▉| 483/485 [01:07<00:00,  6.98it/s]





100%|█████████▉| 484/485 [01:07<00:00,  7.03it/s]

same line space - True
241.575171873
242
False
False
same line space - True
533.950958852
500
False
False
spacings - True
292.375786979
258
angles - True
-0.0490117689861
-0.421516815161
curves - False
-3.36990547814e-06
0.000717215120319
left not updated
spacings - True
-292.375786979
-258
angles - True
-0.421516815161
-0.0490117689861
curves - False
0.000717215120319
-3.36990547814e-06
right not updated


same line space - True
241.575171873
242
False
False
same line space - True
540.144628471
500
False
False
spacings - True
298.569456598
258
angles - True
-0.0490117689861
-0.477699666526
curves - False
-3.36990547814e-06
0.000793154307013
left not updated
spacings - True
-298.569456598
-258
angles - True
-0.477699666526
-0.0490117689861
curves - False
0.000793154307013
-3.36990547814e-06
right not updated








100%|██████████| 485/485 [01:07<00:00,  6.94it/s]







[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge.mp4 



In [186]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [187]:
reset_lines()
harder_challenge_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(harder_challenge_output, audio=False)

updated!
updated!
spacings - True
200.930030421
201
angles - True
0.073999941779
0.103634476476
curves - True
-5.07643964088e-05
-9.14898764436e-05
updated!
spacings - True
-200.930030421
-201
angles - True
0.103634476476
0.073999941779
curves - True
-9.14898764436e-05
-5.07643964088e-05
updated!
[MoviePy] >>>> Building video harder_challenge_video_output.mp4
[MoviePy] Writing video harder_challenge_video_output.mp4








  0%|          | 0/1200 [00:00<?, ?it/s]





  0%|          | 1/1200 [00:00<02:43,  7.32it/s]

same line space - True
213.7625991
213
False
False
same line space - True
416.402667198
414
False
False
spacings - True
202.640068098
201
angles - True
0.073999941779
0.0973152430352
curves - True
-5.07643964088e-05
-7.99665916773e-05
updated!
spacings - True
-202.640068098
-201
angles - True
0.0973152430352
0.073999941779
curves - True
-7.99665916773e-05
-5.07643964088e-05
updated!


same line space - True
213.216731452
213
False
False
same line space - True
418.23138684
415
False
False
spacings - True
205.014655388
202
angles - True
0.0728630607717
0.0802203418732
curves - True
-4.47817976583e-05
-6.19091523564e-05
updated!
spacings - True
-205.014655388
-202
angles - True
0.0802203418732
0.0728630607717
curves - True
-6.19091523564e-05
-4.47817976583e-05
updated!








  0%|          | 2/1200 [00:00<02:50,  7.02it/s]

...

same line space - True
233.264699335
233
False
False
same line space - True
428.83029317
430
False
False
spacings - True
195.565593835
197
angles - True
0.16737465485
0.117526610245
curves - True
-0.00015391687934
-8.27492351696e-05
updated!
spacings - True
-195.565593835
-197
angles - True
0.117526610245
0.16737465485
curves - True
-8.27492351696e-05
-0.00015391687934
updated!
same line space - True
233.556526645
233
False
False
same line space - True
426.736684816
430
False
False
spacings - True
193.180158171
197
angles - True
0.181010017679
0.139907068575
curves - True
-0.000163399397551
-0.000102366514486
updated!
spacings - True
-193.180158171
-197
angles - True
0.139907068575
0.181010017679
curves - True
-0.000102366514486
-0.000163399397551
updated!








100%|█████████▉| 1197/1200 [03:03<00:00,  6.29it/s]





100%|█████████▉| 1198/1200 [03:03<00:00,  6.45it/s]

same line space - True
233.773010566
233
False
False
same line space - True
425.098767445
429
False
False
spacings - True
191.325756879
196
angles - True
0.189810672549
0.175896799371
curves - True
-0.000169060572616
-0.00013495984294
updated!
spacings - True
-191.325756879
-196
angles - True
0.175896799371
0.189810672549
curves - True
-0.00013495984294
-0.000169060572616
updated!
same line space - True
233.759729781
233
False
False
same line space - True
425.123957652
429
False
False
spacings - True
191.364227871
196
angles - True
0.204688585878
0.194975035153
curves - True
-0.000181202468111
-0.000147784098212
updated!
spacings - True
-191.364227871
-196
angles - True
0.194975035153
0.204688585878
curves - True
-0.000147784098212
-0.000181202468111
updated!








100%|█████████▉| 1199/1200 [03:03<00:00,  6.40it/s]







[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_output.mp4 



In [188]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))